# neuralmt: default program

In [ ]:
from default import *
import os, sys

## Run the default solution on dev

In [ ]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [06:11,  3.51it/s]

i had my own , , i had my own . . 
<unk> was <unk> <unk> , and <unk> <unk> . 
<unk> was a <unk> <unk> . 
and she was , and she was <unk> , and she was <unk> , and she was <unk> , and she <unk> , and she was <unk> , and she <unk> , and she <unk> , and she <unk> . 
i was , i was , i was , i was . 
and so , a first - <unk> , , a <unk> , a <unk> . 
and i was going to talk about the , and i wanted to talk about the <unk> . 
i i , i i , i . 
i did n't have it . 
and , the the , , , the , , , was to the <unk> , , and the <unk> , and the <unk> , and the <unk> , and the <unk> , and it was <unk> to 
i was , , " , " , " , " , " and , " 
later came later , later came later later came later later came later later came later later later came later later later later later later later later later later later later later later later later later later later later later later later later later later later later . 
and i did n't like , like , and i did n't have a <unk> like 
i was to <unk> my <unk> , <unk

## Evaluate the default output

In [ ]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 2.49 28.3/5.4/1.3/0.4 (BP = 0.854 ratio = 0.864 hyp_len = 21503 ref_len = 24902)


## Documentation

Write some beautiful documentation of your program here.

### Baseline

We implement the baseline method according to the description on the website. And then the dev score rises to 17.1139 as described on the website.

In [ ]:
from neuralmt import *
import os, sys

/home/zerono614/extra/nlpclass-1207-g-red/hw4/venv/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


#### Fix the function ***calcAlpha***

In [ ]:
# Start working from here, both 'calcAlpha' and 'forward' need to be fixed
def calcAlpha(self, decoder_hidden, encoder_out):
    """
    param encoder_out: (seq, batch, dim),
    param decoder_hidden: (seq, batch, dim)
    """
    seq, batch, dim = encoder_out.shape
    # scores = torch.Tensor([seq * [batch * [1]]]).permute(2, 1, 0)
    # alpha = torch.nn.functional.softmax(scores, dim=1)
    W_enc = self.W_enc(encoder_out)
    W_dec = self.W_dec(decoder_hidden)
    score = W_enc + W_dec
    score = self.V_att(torch.tanh(score))
    score = score.permute(2, 1, 0)
    alpha = torch.nn.functional.softmax(score, dim=-1)
    return alpha

#### Fix the function ***forward***

In [ ]:
def forward(self, decoder_hidden, encoder_out):
        """
        encoder_out: (seq, batch, dim),
        decoder_hidden: (seq, batch, dim)
        """
        alpha = self.calcAlpha(decoder_hidden, encoder_out)
        seq, _, dim = encoder_out.shape
        encoder_out = encoder_out.permute(1, 0, 2)
        # context = (torch.sum(encoder_out, dim=0) / seq).reshape(1, 1, dim)
        context = torch.matmul(alpha, encoder_out).reshape(_, 1, dim)
        return context, alpha

### Beam search decoding

We use beam search decoding to improve the performance of this NMT. According to the description of beam search decoding and the size of our text, we decide to use `queue` to store the top k candidate word since if we use `list` to store those words, then we can not store other information such as the `decoder_hiddens` of every step.  

In [ ]:
def greedyDecoder(decoder, encoder_out, encoder_hidden, maxLen,
                  eos_index):
    seq1_len, batch_size, _ = encoder_out.size()
    target_vocab_size = decoder.target_vocab_size

    outputs = torch.autograd.Variable(
        encoder_out.data.new(maxLen, batch_size, target_vocab_size))
    alphas = torch.zeros(maxLen, batch_size, seq1_len)
    # take what we need from encoder
    decoder_hidden = encoder_hidden[-decoder.n_layers:]
    # start token (ugly hack)
    output = torch.autograd.Variable(
        outputs.data.new(1, batch_size).fill_(eos_index).long())

    # The width of beam search decoding
    topk = 30
    sequences = collections.deque()
    emptyNode = Node(None, output, 0, decoder_hidden)
    sequences.append(emptyNode)

    for t in range(maxLen):
        scores = []
        decoder_hiddens = []
        parentNodes = []
        for j in range(len(sequences)):
            current = sequences.pop()
            parentNodes.append(current)
            decoder_hidden = current.decoder_hidden
            output = current.word

            score, decoder_hidden, _ = decoder(output, encoder_out, decoder_hidden)
            score = score.squeeze()
            score = torch.log(torch.nn.functional.softmax(score, dim=-1))
            scores.append(score + current.score)
            decoder_hiddens.append(decoder_hidden)
        scores = torch.stack(scores)
        decoder_hiddens = torch.stack(decoder_hiddens)

        topk_scores = scores.view(-1).data.topk(topk)
        topk_bins = topk_scores[1] / len(scores[0])
        topk_words = topk_scores[1] % len(scores[0])

        for k in range(topk):
            parent = int(topk_bins[k])

            new_parent = parentNodes[parent]
            output = topk_words[k].reshape(1, 1)
            node = Node(new_parent, output, topk_scores[0][k], decoder_hiddens[parent])
            sequences.append(node)

    best = sequences[0]
    outputs = []
    outputs.append(best.word)
    parent = best.parent
    while parent != None:
        outputs.append(parent.word)
        parent = parent.parent
    outputs = torch.flip(torch.LongTensor(outputs), [0])
    outputs = outputs[1:51].reshape(50, 1, 1)

    return outputs

#### Also modify the function `translate` to adjust the outputs

This part will be explain in the analysis part.

In [ ]:
def translate(model, test_iter):
    results = []
    for i, batch in tqdm(enumerate(test_iter)):
        output = model(batch.src)
        output = model.tgt2txt(output[:, 0].data).strip().split('<EOS>')[0]
        results.append(output)

    # Eliminate the consecutive words that are the same
    # Replace <unk> with the most common word "the" 
    prev_w= None
    newSeq = []
    for sentence in results:
        new_sentence = []
        # Eliminate the consecutive words that are the same
        for w in sentence.split(" "):
            if prev_w != None:
                if w == prev_w:
                    prev_w = w
                    continue
            # Replace <unk> with the most common word "the" 
            if w == '<unk>':
                if prev_w != None:
                    if prevWord != "the":
                        w = "the"
            prev_w = w
            new_sentence.append(w)
        newSeq.append(" ".join(new_sentence))
    return newSeq


## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

### Run the baseline solution on dev

In [ ]:
model = Seq2Seq(build=False)
model.load(os.path.join('..', 'data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('..', 'data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

/home/zerono614/extra/nlpclass-1207-g-red/hw4/venv/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
0it [00:00, ?it/s]/home/zerono614/extra/nlpclass-1207-g-red/hw4/venv/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
1305it [03:29,  6.24it/s]

when when i was in my 20s , i had my first <unk> . . 
i was a and i was particularly in berkeley . berkeley . 
she was a a woman woman named alex . 
when the came came up the first first , she came up and she a a a , and she fell into the couch in my office , <unk> her her and and told me , about about about . 
and when i heard this , i was was . 
my my , , had a a patient patient for a first patient . 
and i got a woman in the people who wanted to talk about about . 
i 'll leave that , i thought , . 
but i did n't have it . 
with the the stories , that alex opened into the , was easy easy to me just to with with head , while we were the problems . 
" 30 is the new 20 " , " alex , and as far as i did , i had right . 
work came later later , later came later later , later came later later later later came later later . 
people people in the the , and alex , and i did n't have time . 
but soon my my my was , my head , to ask . 
i i was going to . 
i said , " yeah , you know , you 're wit

In [ ]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('..', 'data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results).score)

17.11393268339238


### Run the baseline + beam search decoding solution on dev

In [ ]:
from neuralmt import *
import os, sys

In [ ]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [1:35:40,  4.40s/it]

when when i was in my 20s , i had my first <unk> . . 
i 've been up and studied clinical in berkeley . berkeley . 
she was a woman woman named alex . 
as soon as the first arrived came , , she came up and she a a , and she fell into the couch in my office , <unk> her , and she told me to talk about about <unk> . 
and when i heard this , i was <unk> . 
i mean , my <unk> had had a a patient patient as a first patient . 
and i got a woman in the people who wanted to talk about about . . 
i 'll leave it , i i thought . 
but i did n't have it . 
with the the stories stories that brought out into the back , it was easy to me , just to deal with head , as we faced the problems . . 
" 30 is the new 20 " said , " alex , and as far as i did , she had right . 
work came later later , later came later , later came later later , later came later later . 
people people in the , like alex , and i did n't have time time . 
but soon became my god , alex <unk> in question . 
i was against . . 
i said , 

In [ ]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results).score)

19.588472057841773


### Problem we met during improving the performance

According to the description of beam search decoding, we decide to increase the width of beam search to get a better dev score. We got around `19.602` when we use 30 as the width. However, when we increase the width from 30 to 60, the score is `19.588472057841773`, which is almost the same as the score with the width of 30. 

1. After checking the result, we found that it is very strange that some sentences have consecutive words that are exactly the same. Although the situation results from our algorithm, it is abnormal to have this situation in the real word. We decide to adjust this situation after decoding. 

Strange situation
```
when when i was in my 20s , i had my first <unk> . . 

```

Normal situation
```
when i was in my 20s , i had my first <unk> . . 

```

2. And we also find that `<unk>` actually has a worse effect on our program. 

After fixing all these problems and selecting 25 as the width, we got a dev score of `20.028452299380888`

The only puzzle we still have is that we do not know why the performance can not increase when we increase the width of beam search. We think this is might be that the correct word actually not be included in the candidate words, thus it is impossible to have the correct word in the sentence. 

In [2]:
from neuralmt import *
import os, sys

In [3]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [29:00,  1.33s/it]

when i was in my 20s , i had my first the . 
i was a , and i studied in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up , and she was a , and she fell into my office in my office , the her , and she told me to talk about the . 
and when i heard this , i was the . 
i mean , my the had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , just to deal with head , as we faced the problems . 
" 30 is the new 20 " , " alex , and as far as i did , they had right . 
work came later , later came later , later came later , later came later . 
people in the , like alex , and i did n't have time . 
but soon , my , my god was asking in question . 
i was holding . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry it . " 
and he said , " well , "

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results).score)

20.028452299380888
